In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import random
from transformers import BertTokenizer, BertModel
import json
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
torch.manual_seed(1)

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cpu"

In [3]:
# #check if cuda is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

In [4]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

#### Data Loading

In [5]:
train_data = json.load(open('../Dataset/BIO_Tagged/ATE_train.json', 'r'))
test_data = json.load(open('../Dataset/BIO_Tagged/ATE_test.json', 'r'))
val_data = json.load(open('../Dataset/BIO_Tagged/ATE_val.json', 'r'))

In [6]:
word_to_idx = pickle.load(open('../Utils/word_to_idx.pkl', 'rb'))

In [7]:
tag_to_ix = pickle.load(open('../Utils/tag_to_ix.pkl', 'rb'))

#### RNN model

In [8]:
class RNN_model(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, device='cpu'):
        super(RNN_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_mat)).to(device)
        self.rnn = nn.RNN(embedding_dim, hidden_dim).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)
        self.start_tag = start_tag
        self.end_tag = end_tag
        self.tag_to_ix = tag_to_ix
        self.target_size = target_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        rnn_out, _ = self.rnn(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(rnn_out.view(len(sentence), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores


#### LSTM Model

In [9]:
class LSTM_model(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, device='cpu'):
        super(LSTM_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_mat)).to(device)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)
        self.start_tag = start_tag
        self.end_tag = end_tag
        self.tag_to_ix = tag_to_ix
        self.target_size = target_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

#### GRU model

In [10]:
class GRU_model(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, device='cpu'):
        super(GRU_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_mat)).to(device)
        self.gru = nn.GRU(embedding_dim, hidden_dim).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)
        self.start_tag = start_tag
        self.end_tag = end_tag
        self.tag_to_ix = tag_to_ix
        self.target_size = target_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        gru_out, _ = self.gru(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(gru_out.view(len(sentence), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

#### Embedding mats

In [11]:
bert_embedding_mat = pickle.load(open('../Extracted Word Embeddings/legal_bert_embedding_mat.pkl', 'rb'))
word2vec_embedding_mat = pickle.load(open('../Extracted Word Embeddings/word2vec_embedding_mat.pkl', 'rb'))
glove_embedding_mat = pickle.load(open('../Extracted Word Embeddings/glove_embedding_mat.pkl', 'rb'))

#### Glove + RNN 

In [12]:
rnn_model = RNN_model(len(word_to_idx), 300, 256, len(tag_to_ix), glove_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    train_loss_temp = 0
    val_loss_temp = 0
    train_f1_temp = 0
    val_f1_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        train_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp/len(train_data))
    
    with torch.no_grad():
        rnn_model.eval()
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            val_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    val_loss.append(val_loss_temp/len(val_data))
    val_f1.append(val_f1_temp/len(val_data))

    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 219/219 [00:00<00:00, 641.54it/s]


Train Loss: 0.5968134191882268, Val Loss: 0.4906317757008827, Train F1: 0.459286623513282, Val F1: 0.5663856508567454
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 527.77it/s]


Train Loss: 0.47993985639173725, Val Loss: 0.44160774785634044, Train F1: 0.5662979265034384, Val F1: 0.6233874019219469
Epoch:  2


...

Train Loss: 0.363856319527226, Val Loss: 0.4034735304920096, Train F1: 0.6849642588034722, Val F1: 0.6588760977371073
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 614.58it/s]


Train Loss: 0.3546119403447668, Val Loss: 0.40396422222596867, Train F1: 0.6928735673691673, Val F1: 0.6623555267227851
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 391.28it/s]

Train Loss: 0.34607392498528455, Val Loss: 0.40475360673244143, Train F1: 0.6998949622485837, Val F1: 0.6630402886840052


In [13]:
torch.save(rnn_model, 'Non Trainable Embeddings/Glove+RNN/model.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove+RNN/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove+RNN/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Glove+RNN/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Glove+RNN/val_f1.pkl', 'wb'))

#### Word2vec + RNN

In [15]:
rnn_model = RNN_model(len(word_to_idx), 300, 256, len(tag_to_ix), word2vec_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    train_loss_temp = 0
    val_loss_temp = 0
    train_f1_temp = 0
    val_f1_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        train_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp/len(train_data))
    
    with torch.no_grad():
        rnn_model.eval()
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            val_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    val_loss.append(val_loss_temp/len(val_data))
    val_f1.append(val_f1_temp/len(val_data))

    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  0%|          | 0/906 [00:00<?, ?it/s]

100%|██████████| 219/219 [00:00<00:00, 466.24it/s]


Train Loss: 0.6535127756019302, Val Loss: 0.5358878160448379, Train F1: 0.38059835866048025, Val F1: 0.4678420440307875
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 478.74it/s]


...

Train Loss: 0.3447088405744853, Val Loss: 0.35916402391647095, Train F1: 0.6980137923857486, Val F1: 0.7010041891924879
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 425.41it/s]


Train Loss: 0.3354455349806523, Val Loss: 0.3579895687906165, Train F1: 0.7069530224102186, Val F1: 0.7000983633459458
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 410.26it/s]

Train Loss: 0.3271152251636482, Val Loss: 0.35792350151563346, Train F1: 0.715636726258825, Val F1: 0.7013391906011901


In [16]:
torch.save(rnn_model, 'Non Trainable Embeddings/Word2vec+RNN/model.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec+RNN/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec+RNN/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Word2vec+RNN/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Word2vec+RNN/val_f1.pkl', 'wb'))

#### Bert + RNN

In [17]:
rnn_model = RNN_model(len(word_to_idx), 768, 512, len(tag_to_ix), bert_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    train_loss_temp = 0
    val_loss_temp = 0
    train_f1_temp = 0
    val_f1_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        train_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp/len(train_data))
    
    with torch.no_grad():
        rnn_model.eval()
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            val_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    val_loss.append(val_loss_temp/len(val_data))
    val_f1.append(val_f1_temp/len(val_data))

    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  1%|          | 9/906 [00:00<00:10, 83.66it/s]

100%|██████████| 219/219 [00:00<00:00, 492.30it/s]


Train Loss: 0.6838453699717458, Val Loss: 0.6475972284192908, Train F1: 0.4108339289631352, Val F1: 0.4209117892116244
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 449.31it/s]


...

Train Loss: 0.6184745836369776, Val Loss: 0.614347759583225, Train F1: 0.46164803585873876, Val F1: 0.449163102256526
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 290.69it/s]


Train Loss: 0.6145513336159799, Val Loss: 0.6122919484360577, Train F1: 0.46372080395479465, Val F1: 0.4560875929647532
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 430.22it/s]

Train Loss: 0.6107513507865122, Val Loss: 0.6106750203049891, Train F1: 0.4659139108601753, Val F1: 0.4580197127836402


In [18]:
torch.save(rnn_model, 'Non Trainable Embeddings/Bert+RNN/model.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert+RNN/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert+RNN/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Bert+RNN/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Bert+RNN/val_f1.pkl', 'wb'))

#### Glove + GRU

In [19]:
gru_model = GRU_model(len(word_to_idx), 300, 256, len(tag_to_ix), glove_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(gru_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    train_loss_temp = 0
    val_loss_temp = 0
    train_f1_temp = 0
    val_f1_temp = 0
    gru_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        gru_model.zero_grad()
        tag_scores = gru_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        train_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp/len(train_data))
    
    with torch.no_grad():
        gru_model.eval()
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = gru_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            val_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    val_loss.append(val_loss_temp/len(val_data))
    val_f1.append(val_f1_temp/len(val_data))

    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  0%|          | 0/906 [00:00<?, ?it/s]

100%|██████████| 219/219 [00:00<00:00, 265.88it/s]


Train Loss: 0.6877748227079973, Val Loss: 0.5866534595620142, Train F1: 0.36627919093451694, Val F1: 0.4462616602840087
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 359.10it/s]


...

Train Loss: 0.4199479476571346, Val Loss: 0.4163357388891586, Train F1: 0.62394911283692, Val F1: 0.6382639654823741
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 351.58it/s]


Train Loss: 0.4098619041063928, Val Loss: 0.41023150481045517, Train F1: 0.6353051317016546, Val F1: 0.6432222298997439
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 283.26it/s]

Train Loss: 0.40089639766392593, Val Loss: 0.4049899061656978, Train F1: 0.6436106278633325, Val F1: 0.639819831002705


In [20]:
torch.save(gru_model, 'Non Trainable Embeddings/Glove+GRU/model.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove+GRU/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove+GRU/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Glove+GRU/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Glove+GRU/val_f1.pkl', 'wb'))

#### Word2vec + GRU

In [21]:
gru_model = GRU_model(len(word_to_idx), 300, 256, len(tag_to_ix), word2vec_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(gru_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    train_loss_temp = 0
    val_loss_temp = 0
    train_f1_temp = 0
    val_f1_temp = 0
    gru_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        gru_model.zero_grad()
        tag_scores = gru_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        train_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp/len(train_data))
    
    with torch.no_grad():
        gru_model.eval()
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = gru_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            val_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    val_loss.append(val_loss_temp/len(val_data))
    val_f1.append(val_f1_temp/len(val_data))

    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  0%|          | 0/906 [00:00<?, ?it/s]

100%|██████████| 219/219 [00:00<00:00, 339.64it/s]


Train Loss: 0.7604094644058619, Val Loss: 0.6435012930332253, Train F1: 0.35151441913565007, Val F1: 0.3531173110138638
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 318.17it/s]


...

Train Loss: 0.42143394626167174, Val Loss: 0.41049077161098724, Train F1: 0.6088956180109768, Val F1: 0.6260205462626923
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 324.99it/s]


Train Loss: 0.41131051442283667, Val Loss: 0.4019248680505034, Train F1: 0.6190169581061779, Val F1: 0.6440093282600707
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 229.34it/s]

Train Loss: 0.40258057483741255, Val Loss: 0.3946731783891922, Train F1: 0.6280850830387626, Val F1: 0.6592141532923255


In [22]:
torch.save(gru_model, 'Non Trainable Embeddings/Word2vec+GRU/model.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec+GRU/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec+GRU/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Word2vec+GRU/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Word2vec+GRU/val_f1.pkl', 'wb'))

#### Bert + GRU

In [23]:
gru_model = GRU_model(len(word_to_idx), 768, 512, len(tag_to_ix), bert_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(gru_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    train_loss_temp = 0
    val_loss_temp = 0
    train_f1_temp = 0
    val_f1_temp = 0
    gru_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        gru_model.zero_grad()
        tag_scores = gru_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        train_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp/len(train_data))
    
    with torch.no_grad():
        gru_model.eval()
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = gru_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            val_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    val_loss.append(val_loss_temp/len(val_data))
    val_f1.append(val_f1_temp/len(val_data))

    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  0%|          | 0/906 [00:00<?, ?it/s]

100%|██████████| 219/219 [00:01<00:00, 174.76it/s]


Train Loss: 0.6987751507206469, Val Loss: 0.6678536323379708, Train F1: 0.3748922840792369, Val F1: 0.3927246398718795
Epoch:  1


100%|██████████| 219/219 [00:01<00:00, 156.65it/s]


...

Train Loss: 0.6366377842610509, Val Loss: 0.6342574647844654, Train F1: 0.44616485362484753, Val F1: 0.44079209455464113
Epoch:  8


100%|██████████| 219/219 [00:01<00:00, 160.54it/s]


Train Loss: 0.6331809836770789, Val Loss: 0.632328130066667, Train F1: 0.4495940085047949, Val F1: 0.4442398268555838
Epoch:  9


100%|██████████| 219/219 [00:01<00:00, 152.92it/s]

Train Loss: 0.6299121094657886, Val Loss: 0.6306091001316837, Train F1: 0.45096639102456526, Val F1: 0.44672981845427473


In [24]:
torch.save(gru_model, 'Non Trainable Embeddings/Bert+GRU/model.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert+GRU/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert+GRU/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Bert+GRU/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Bert+GRU/val_f1.pkl', 'wb'))

#### Glove + LSTM

In [25]:
lstm_model = LSTM_model(len(word_to_idx), 300, 256, len(tag_to_ix), glove_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    train_loss_temp = 0
    val_loss_temp = 0
    train_f1_temp = 0
    val_f1_temp = 0
    lstm_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        lstm_model.zero_grad()
        tag_scores = lstm_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        train_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp/len(train_data))
    
    with torch.no_grad():
        lstm_model.eval()
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = lstm_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            val_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    val_loss.append(val_loss_temp/len(val_data))
    val_f1.append(val_f1_temp/len(val_data))

    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 219/219 [00:00<00:00, 342.00it/s]


Train Loss: 0.7816405328562192, Val Loss: 0.6583317537285965, Train F1: 0.3515741671625853, Val F1: 0.3531173110138638
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 400.92it/s]


Train Loss: 0.6441842318568008, Val Loss: 0.604199711182346, Train F1: 0.35803346825563187, Val F1: 0.38346339543386804
Epoch:  2


...

Train Loss: 0.4664415038020142, Val Loss: 0.4456977086932692, Train F1: 0.5790176834997439, Val F1: 0.5950062855061207
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 364.73it/s]


Train Loss: 0.45341039362628727, Val Loss: 0.4345080314297654, Train F1: 0.5877747024987416, Val F1: 0.6027366631881801
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 372.02it/s]

Train Loss: 0.4419508746292681, Val Loss: 0.4251132215879279, Train F1: 0.5954390516435106, Val F1: 0.6057877371685894


In [26]:
torch.save(lstm_model, 'Non Trainable Embeddings/Glove+LSTM/model.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove+LSTM/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove+LSTM/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Glove+LSTM/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Glove+LSTM/val_f1.pkl', 'wb'))

#### Word2vec + LSTM

In [27]:
lstm_model = LSTM_model(len(word_to_idx), 300, 256, len(tag_to_ix), word2vec_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    train_loss_temp = 0
    val_loss_temp = 0
    train_f1_temp = 0
    val_f1_temp = 0
    lstm_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        lstm_model.zero_grad()
        tag_scores = lstm_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        train_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp/len(train_data))
    
    with torch.no_grad():
        lstm_model.eval()
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = lstm_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            val_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    val_loss.append(val_loss_temp/len(val_data))
    val_f1.append(val_f1_temp/len(val_data))

    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 219/219 [00:00<00:00, 288.10it/s]


Train Loss: 0.8449351207310001, Val Loss: 0.6961075300223207, Train F1: 0.3483793784747095, Val F1: 0.3531173110138638
Epoch:  1


100%|██████████| 219/219 [00:00<00:00, 355.27it/s]


Train Loss: 0.6822504956793312, Val Loss: 0.6535659399751115, Train F1: 0.35221363021064966, Val F1: 0.3531173110138638
Epoch:  2


...

Train Loss: 0.5089066948980159, Val Loss: 0.48845615146094806, Train F1: 0.5090672007879095, Val F1: 0.5473228506226737
Epoch:  8


100%|██████████| 219/219 [00:00<00:00, 339.19it/s]


Train Loss: 0.4892592430937106, Val Loss: 0.47070595696908696, Train F1: 0.5413926229152894, Val F1: 0.5652154331093707
Epoch:  9


100%|██████████| 219/219 [00:00<00:00, 375.02it/s]


Train Loss: 0.472696417412221, Val Loss: 0.4556077920818982, Train F1: 0.557499713473176, Val F1: 0.5850595774745727


In [28]:
torch.save(lstm_model, 'Non Trainable Embeddings/Word2vec+LSTM/model.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec+LSTM/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec+LSTM/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Word2vec+LSTM/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Word2vec+LSTM/val_f1.pkl', 'wb'))

#### Bert + LSTM

In [29]:
lstm_model = LSTM_model(len(word_to_idx), 768, 512, len(tag_to_ix), bert_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    train_loss_temp = 0
    val_loss_temp = 0
    train_f1_temp = 0
    val_f1_temp = 0
    lstm_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        lstm_model.zero_grad()
        tag_scores = lstm_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        train_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp/len(train_data))
    
    with torch.no_grad():
        lstm_model.eval()
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = lstm_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            val_f1_temp += f1_score(targets.detach().numpy(), torch.argmax(tag_scores, dim=1).detach().numpy(), average='macro')
    val_loss.append(val_loss_temp/len(val_data))
    val_f1.append(val_f1_temp/len(val_data))

    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 219/219 [00:01<00:00, 157.20it/s]


Train Loss: 0.7167522716963002, Val Loss: 0.676812830717052, Train F1: 0.3547878682102667, Val F1: 0.35788249528469185
Epoch:  1


100%|██████████| 219/219 [00:01<00:00, 127.54it/s]


Train Loss: 0.6783066020073744, Val Loss: 0.6663340824107601, Train F1: 0.36790737678011937, Val F1: 0.38888443691028063
Epoch:  2


...

Train Loss: 0.6465524618100646, Val Loss: 0.6410244481476475, Train F1: 0.43197066002060525, Val F1: 0.4353229748345722
Epoch:  8


100%|██████████| 219/219 [00:01<00:00, 140.58it/s]


Train Loss: 0.6439385161330106, Val Loss: 0.6388820456043226, Train F1: 0.433782664336745, Val F1: 0.4356017746607418
Epoch:  9


100%|██████████| 219/219 [00:01<00:00, 152.38it/s]

Train Loss: 0.6414778806910609, Val Loss: 0.6368587898881468, Train F1: 0.4376682314077051, Val F1: 0.4383686839631542


In [30]:
torch.save(lstm_model, 'Non Trainable Embeddings/Bert+LSTM/model.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert+LSTM/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert+LSTM/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Bert+LSTM/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Bert+LSTM/val_f1.pkl', 'wb'))